## SRJ - GSD 

In [1]:
import numpy as np
import pennylane as qml

def generalized_singles(wires, delta_sz):
    r"""Return generalized single excitation terms

    .. math::
        \hat{T_1} = \sum_{pq} t_{p}^{q} \hat{c}^{\dagger}_{q} \hat{c}_{p}

    """
    sz = np.array(
        [0.5 if (i % 2 == 0) else -0.5 for i in range(len(wires))]
    )  # alpha-beta electrons
    gen_singles_wires = []
    for r in range(len(wires)):
        for p in range(len(wires)):
            if sz[p] - sz[r] == delta_sz and p != r:
                if r < p:
                    gen_singles_wires.append(wires[r : p + 1])
                else:
                    gen_singles_wires.append(wires[p : r + 1][::-1])
    return gen_singles_wires

u = generalized_singles(wires = [0,1,2,3,4,5,6,7], delta_sz= 0)

print(u)


GSD_singles = [[sublist[0], sublist[-1]] for sublist in u]
print('GSD singles',GSD_singles)
print('Length of GSD singles', len(GSD_singles))


[[0, 1, 2], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5, 6], [1, 2, 3], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6, 7], [2, 1, 0], [2, 3, 4], [2, 3, 4, 5, 6], [3, 2, 1], [3, 4, 5], [3, 4, 5, 6, 7], [4, 3, 2, 1, 0], [4, 3, 2], [4, 5, 6], [5, 4, 3, 2, 1], [5, 4, 3], [5, 6, 7], [6, 5, 4, 3, 2, 1, 0], [6, 5, 4, 3, 2], [6, 5, 4], [7, 6, 5, 4, 3, 2, 1], [7, 6, 5, 4, 3], [7, 6, 5]]
GSD singles [[0, 2], [0, 4], [0, 6], [1, 3], [1, 5], [1, 7], [2, 0], [2, 4], [2, 6], [3, 1], [3, 5], [3, 7], [4, 0], [4, 2], [4, 6], [5, 1], [5, 3], [5, 7], [6, 0], [6, 2], [6, 4], [7, 1], [7, 3], [7, 5]]
Length of GSD singles 24


In [2]:
def generalized_pair_doubles(wires):
    r"""Return pair coupled-cluster double excitations

    .. math::
        \hat{T_2} = \sum_{pq} t_{p_\alpha p_\beta}^{q_\alpha, q_\beta}
               \hat{c}^{\dagger}_{q_\alpha} \hat{c}^{\dagger}_{q_\beta} \hat{c}_{p_\beta} \hat{c}_{p_\alpha}

    """
    pair_gen_doubles_wires = [
        [
            wires[r : r + 2],
            wires[p : p + 2],
        ]  # wires for [wires[r], wires[r+1], wires[p], wires[p+1]] terms
        for r in range(0, len(wires) - 1, 2)
        for p in range(0, len(wires) - 1, 2)
        if p != r  # remove redundant terms
    ]
    return pair_gen_doubles_wires

tdoubles = generalized_pair_doubles(wires = [0,1,2,3,4,5,6,7])
print((tdoubles))
print(len(tdoubles))

GSD_doubles = [[sublist[0][0], sublist[0][-1], sublist[1][0], sublist[1][-1]] for sublist in tdoubles]
print('GSD doubles', GSD_doubles)
print('length of GSD doubles', len(GSD_doubles))   

[[[0, 1], [2, 3]], [[0, 1], [4, 5]], [[0, 1], [6, 7]], [[2, 3], [0, 1]], [[2, 3], [4, 5]], [[2, 3], [6, 7]], [[4, 5], [0, 1]], [[4, 5], [2, 3]], [[4, 5], [6, 7]], [[6, 7], [0, 1]], [[6, 7], [2, 3]], [[6, 7], [4, 5]]]
12
GSD doubles [[0, 1, 2, 3], [0, 1, 4, 5], [0, 1, 6, 7], [2, 3, 0, 1], [2, 3, 4, 5], [2, 3, 6, 7], [4, 5, 0, 1], [4, 5, 2, 3], [4, 5, 6, 7], [6, 7, 0, 1], [6, 7, 2, 3], [6, 7, 4, 5]]
length of GSD doubles 12


In [ ]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 2.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886 
r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
        elif len(ash_excitation) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in GSD_singles]
    print('op1 is', op1[0])
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in GSD_doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
            counts_ind = measure(states[-1])
            print('The individula states counts', counts_ind)
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                #print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                S[i,j] = left_op.dot(right_op).real
     
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
          
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state
